In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img





In [20]:
def get_file_paths(path):
    '''
    Takes a folder path and returns a set of all file paths of .jpg in the folders
    Input: Folder path
    '''
    file_set = set()

    for direct, _, files in os.walk(path):
        for file_name in files:
            rel_dir = os.path.relpath(direct, path)
            rel_file = os.path.join(rel_dir, file_name)
            if '.jpg' not in rel_file:
                continue
            file_set.add(str(path)+rel_file)

    return file_set

def one_hotify(y, n_classes=None):
    '''Convert array of integers to one-hot format;
    The new dimension is added at the end.'''
    if n_classes is None:
        n_classes = max(y) + 1
    labels = np.arange(n_classes)
    y = y[..., None]
    return (y == labels).astype(int)

def load_images(path,size=(256,256),target='not'):
    
    file_paths = get_file_paths(path)
    
    images = []
    y = []
    for file in file_paths:
        img = keras.preprocessing.image.load_img(file, target_size=size)
        img_arr = keras.preprocessing.image.img_to_array(img)
        images.append(img_arr)
        if target in file.split('/')[-2]:
            y.append(1)
        else:
            y.append(0)
        
    return images, pd.get_dummies(y)

In [21]:
# X2, y2 = load_images(('../data/Train_Val_Test/CASO/train/'))
# X2 = np.array(X2)
print(2)
X3, y3 = load_images('../data/Train_Val_Test/CASO/val/')
X3 = np.array(X3)
print(3)
X, y = load_images('../data/Train_Val_Test/CASO/test/')
X = np.array(X)

2
3


In [22]:
img_width = 256
img_height = 256
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling (we do not want to modify the testing data)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)


# The generator object. 
train_generator = train_datagen.flow_from_directory(
    '../data/Train_Val_Test/CASO/train/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '../data/Train_Val_Test/CASO/val/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 14544 images belonging to 2 classes.
Found 2726 images belonging to 2 classes.


In [23]:
img_width, img_height = 256, 256

nb_train_samples = 14550
nb_validation_samples = 2750
epochs = 5
batch_size = 32
nb_train_samples // batch_size, nb_validation_samples // batch_size

(454, 85)

In [53]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.01)))

In [77]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              )               metrics=['accuracy'])

In [78]:
m = model.fit_generator(
        train_generator,
        steps_per_epoch=25,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=15)

Epoch 1/5
25/25 [==============================] - 182s 7s/step - loss: 0.4797 - accuracy: 0.8256 - val_loss: 0.4759 - val_accuracy: 0.8385
Epoch 2/5
25/25 [==============================] - 178s 7s/step - loss: 0.4695 - accuracy: 0.8375 - val_loss: 0.4541 - val_accuracy: 0.8490
Epoch 3/5
25/25 [==============================] - 194s 8s/step - loss: 0.4645 - accuracy: 0.8388 - val_loss: 0.4403 - val_accuracy: 0.8458
Epoch 4/5
25/25 [==============================] - 195s 8s/step - loss: 0.4594 - accuracy: 0.8363 - val_loss: 0.4450 - val_accuracy: 0.8500
Epoch 5/5
25/25 [==============================] - 187s 8s/step - loss: 0.4378 - accuracy: 0.8531 - val_loss: 0.4454 - val_accuracy: 0.8458


In [74]:
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

In [75]:
base_model.trainable = False

In [76]:
model = keras.Sequential([
    base_model,
    layers.Dense(100),
    layers.Dense(2,
                activation='softmax',
                kernel_regularizer=keras.regularizers.l2(0.01))
])

In [72]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
dense_15 (Dense)             (None, 100)               204900    
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 202       
Total params: 21,066,582
Trainable params: 205,102
Non-trainable params: 20,861,480
_________________________________________________________________
